In [ ]:
%pip install numerapi joblib

In [ ]:
import pandas as pd
from numerapi import CryptoAPI
import joblib

from dotenv import load_dotenv
import os

In [ ]:
# Load the .env file
load_dotenv()

# Access the environment variables
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Use API keys to authenticate
cApi = CryptoAPI(secret_key = api_secret,  public_id = api_key)

cApi.download_dataset("crypto/v1.0/live_universe.parquet", dest_path=output_live_parquet_file_path)


In [ ]:
historical_data = pd.read_csv('../data/historical_data.csv')
historical_data['date'] = pd.to_datetime(historical_data['date'])

live_df = pd.read_parquet(output_live_parquet_file_path)
live_df.reset_index(drop=True, inplace=True)
live_df.to_csv(output_live_csv_file_path)

# Load the trained model
model = joblib.load(input_file_path)

In [ ]:
# Get the latest date available in the historical data
latest_date = historical_data['date'].max()

# Filter historical data for the latest date
latest_data = historical_data[historical_data['date'] == latest_date]

In [ ]:
# Merge live data with the latest historical data
live_data = live_df.merge(latest_data, on='symbol', how='left')

# Select the features
feature_cols = ['open', 'high', 'low', 'close', 'volume']

# Ensure live_data has the necessary features
live_features = live_data[feature_cols]

# Predict using the model
live_data["signal"] = model.predict(live_features)

# Ensure predictions are between 0 and 1
live_data["signal"] = live_data["signal"].rank(pct=True)

# Format and save submission
live_data[['symbol', 'signal']].to_parquet(output_submission_file_path)

In [ ]:
# Get model ids and submit models
models = cApi.get_models()
print(f'Models available: {models}')

model_id = cApi.get_models()['1sttrialmodel']
print('Submitting prediction for {model_id}')
# cApi.upload_predictions(output_submission_file_path, model_id = model_id)

print('Submission completed')